# Creating Fabric Data Agent via Code

In [1]:
%pip install fabric-data-agent-sdk

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 8.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of azure-kusto-data to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 16.7 MB/s eta 0:00:00
Reason for being yanked: Yanked due to conflicts with CVE-2024-35195 mitigation
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.3/699.3 kB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/

#### Imports

In [2]:
from fabric.dataagent.client import (
    FabricDataAgentManagement,
    create_data_agent,
    delete_data_agent,
)

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 9, Finished, Available, Finished)

##### Configs for Data Agent Name and Data Sources

In [15]:
# Configuration
data_agent_name = "projectFinancialsAgentCode"
lakehouse_name = "data_agent_lh"
table_names = [ "BST10_ACTIVE_PROJECTS"
                ,"TPF_DUMP"]

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 22, Finished, Available, Finished)

In [16]:
# Create
data_agent = create_data_agent(data_agent_name)
data_agent = FabricDataAgentManagement(data_agent_name)

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 23, Finished, Available, Finished)

In [17]:
# Update the data agent's configuration with detailed instructions for query expansion and retrieve the updated configuration
data_agent.update_configuration(
    instructions= """
    You are a helpful and focused data agent assisting users querying a Microsoft Fabric Lakehouse.

    The Lakehouse contains two tables:
    1. `TPF_DUMP` — time-series financial metrics at the project level (monthly granularity).
    2. `BST10_ACTIVE_PROJECTS` — metadata about active projects (client, start/end dates, etc.).

    These tables are joined using:
    `TPF_DUMP.PROJECT_CODE = BST10_ACTIVE_PROJECTS.Proj:_Project_Code`

    Important behavior guidelines:
    - Respond only to questions answerable by the data. If a question is ambiguous or not supported, ask for clarification.
    - Format your SQL queries in **T-SQL** syntax, compatible with Microsoft Fabric.
    - Use the join only when required (e.g., when project name or client name is requested).
    - Fiscal period (`FISCPER`) is in `YYYYMM` format and **shifted 6 months forward**. For example, `202304` represents **October 2022**. Consider this when calculating dates.
    - Be aware that `TPF_DUMP` may contain null values and scientific notation (e.g., `0E-9`) — handle these gracefully.
    - Prioritize `TD_` prefixed columns (e.g., `TD_BUDGET_EFFORT`, `TD_VARIANCE`) for effort and variance-based calculations.

    Respond with **concise summaries and well-structured SQL**. Avoid overly technical explanations unless requested. Include fiscal period conversion logic in queries where time context is needed.

    Examples of supported question types:
    - What is the total TD budget effort in a given fiscal period?
    - List all projects with a TD variance greater than a threshold.
    - Show WIP balances for the last 3 fiscal periods.
    - Join with project metadata (name, client) when requested.

    Do not make up fields or relationships that are not present in the schema.

    Tables:
    - TPF_DUMP (metrics)
    - BST10_ACTIVE_PROJECTS (project metadata)

    Default join key: `PROJECT_CODE` = `Proj:_Project_Code`

    Make sure when any date is used, you convert it into its fiscal period form. Only use fiscal period form in questions and answers if the format is YYYYMM.
    """,
)

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 24, Finished, Available, Finished)

In [18]:
def safe_get_datasources(): # add_datasource calls get_datasources, which doesn't handle None types.... This is a modified function which handles this
    config = data_agent._client.get_configuration()
    data_sources = config.value.get("dataSources") or []
    return [data_agent._client.get_datasource(ds["id"]) for ds in data_sources]

data_agent._client.get_datasources = safe_get_datasources

data_agent.add_datasource(lakehouse_name, type="lakehouse")

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 25, Finished, Available, Finished)

Datasource(123aafb5-2b8b-405f-b84d-b60a6e8df469)

In [19]:
datasource = data_agent.get_datasources()[0]

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 26, Finished, Available, Finished)

In [20]:
for table_name in table_names:
    datasource.select("dbo", table_name)

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 27, Finished, Available, Finished)

In [21]:
datasource.pretty_print()

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 28, Finished, Available, Finished)

 dbo
  | TPF_DUMP *
  |  | PROJECT_ID
  |  | PROJECT_CODE
  |  | FISCPER
  |  | WIP_BALANCE
  |  | WIP_AGEING1
  |  | WIP_AGEING2
  |  | WIP_UNALLOCATED
  |  | R_BALANCE
  |  | RB_AGEING1
  |  | RB_AGEING2
  |  | RB_AGEING3
  |  | RB_UNALLOCATED
  |  | VARIANCE
  |  | BUDGET_EFFORT
  |  | EFF_EFFORT_AT_COMP
  |  | TD_BUDGET_EFFORT
  |  | TD_VARIANCE
  |  | TD_EFF_EFFORT_AT_COMP
  |  | TD_PROPOSAL_EFFORT
  | BST10_ACTIVE_PROJECTS *
  |  | Proj:_Active_Status
  |  | Proj:_Base_Currency_Code
  |  | Proj:_Client_Name
  |  | Proj:_Company_Currency
  |  | Proj:_Created_Date
  |  | Proj:_Description
  |  | Proj:_End_Date
  |  | Proj:_Has_Chargeable
  |  | Proj:_Has_Proposal
  |  | Proj:_Project_ID
  |  | Proj:_Market_Segment_Code
  |  | Proj:_Project_Name
  |  | Proj:_Project_Currency
  |  | Proj:_Manager_Code
  |  | Proj:_Project_Code
  |  | Proj:_Service_Line
  |  | Proj:_Start_Date


'data_agent_lh'

Notes for data source

In [22]:
ds_notes ="""
This Lakehouse contains project delivery and financial data from two joined tables. It is used to analyze budget, variance, effort, and WIP metrics across active projects. Data is refreshed monthly and already filtered to only include active projects.

TABLE 1: TPF_DUMP – Time-Series Financial Metrics
Each row represents a project at a specific fiscal period. Contains numeric metrics in string or scientific notation format.

PROJECT_CODE (string): join key

FISCPER (string, format YYYYMM): fiscal period, shifted 6 months into the future (e.g., 202304 = October 2022)

TD_BUDGET_EFFORT (string): to-date budgeted effort

TD_VARIANCE (string): to-date variance

TD_EFF_EFFORT_AT_COMP (string): estimate at completion

WIP_BALANCE, R_BALANCE, VARIANCE (string): financial balances

WIP_AGEING1/2, RB_AGEING1/2/3 (string): aged balances

Notes:

Contains many nulls and "0E-9" (treat as 0)

Use CAST(... AS FLOAT) for aggregations

Use FISCPER for period-based filtering or sorting

TABLE 2: BST10_ACTIVE_PROJECTS – Project Metadata
Each row is a unique active project with descriptive info.

Proj:_Project_Code (string): primary key for joining with TPF_DUMP

Proj:_Project_Name (string): project label

Proj:_Client_Name (string): client or funding agency

Proj:_Start_Date / Proj:_End_Date (date): project duration

Proj:_Service_Line (string): business domain

Proj:_Market_Segment_Code (string): vertical or classification

Proj:_Project_Currency, Proj:_Company_Currency (string): currency codes

Proj:_Manager_Code (string): owner or responsible staff

Proj:_Has_Chargeable, Proj:_Has_Proposal (flag): 1 = true, blank/0 = false

Notes:

All projects are already active

Use for grouping/filtering by client, market, or date range

RELATIONSHIP:
Join TPF_DUMP.PROJECT_CODE = BST10_ACTIVE_PROJECTS.Proj:_Project_Code
(one-to-many, since financials are time-based)

TIPS FOR THE AGENT:

Many numeric fields contain null or "0E-9" → handle with care

Use TD_ columns for effort/variance-related analysis

Use [Proj:_...] columns for metadata grouping or labels

Use FISCPER as the time filter, remembering the 6-month shift

Use CAST for all aggregations on numeric columns

Useful metadata grouping columns:

Proj:_Client_Name

Proj:_Service_Line

Proj:_Market_Segment_Code

Proj:_Start_Date

Proj:_Manager_Code

We can group projects by different details such as by Manager Code, Start Dates etc. pretty much anything in the BST10_ACTIVE_PROJECTS Table
"""


StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 29, Finished, Available, Finished)

In [23]:
datasource.update_configuration(instructions=ds_notes)
datasource.get_configuration()["additional_instructions"]

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 30, Finished, Available, Finished)

'\nThis Lakehouse contains project delivery and financial data from two joined tables. It is used to analyze budget, variance, effort, and WIP metrics across active projects. Data is refreshed monthly and already filtered to only include active projects.\n\nTABLE 1: TPF_DUMP – Time-Series Financial Metrics\nEach row represents a project at a specific fiscal period. Contains numeric metrics in string or scientific notation format.\n\nPROJECT_CODE (string): join key\n\nFISCPER (string, format YYYYMM): fiscal period, shifted 6 months into the future (e.g., 202304 = October 2022)\n\nTD_BUDGET_EFFORT (string): to-date budgeted effort\n\nTD_VARIANCE (string): to-date variance\n\nTD_EFF_EFFORT_AT_COMP (string): estimate at completion\n\nWIP_BALANCE, R_BALANCE, VARIANCE (string): financial balances\n\nWIP_AGEING1/2, RB_AGEING1/2/3 (string): aged balances\n\nNotes:\n\nContains many nulls and "0E-9" (treat as 0)\n\nUse CAST(... AS FLOAT) for aggregations\n\nUse FISCPER for period-based filtering

In [24]:
json_key_pairs_dict = {
    "What is the total TD budget effort across all projects in fiscal period 202304?": "-- Enter SQL query\r\nSELECT \r\n    SUM(CAST(TD_BUDGET_EFFORT AS FLOAT)) AS total_td_budget_effort\r\nFROM TPF_DUMP\r\nWHERE FISCPER = '202304';",
    "List project names and TD variance for all projects with TD variance over 5000 in fiscal period 202305.": "-- Enter SQL query\r\nSELECT \r\n    B.[Proj:_Project_Name],\r\n    A.TD_VARIANCE,\r\n    A.FISCPER\r\nFROM TPF_DUMP A\r\nJOIN BST10_ACTIVE_PROJECTS B\r\n    ON A.PROJECT_CODE = B.[Proj:_Project_Code]\r\nWHERE A.FISCPER = '202305'\r\n  AND CAST(A.TD_VARIANCE AS FLOAT) > 5000;",
    "Show top 5 projects with the highest WIP balance in fiscal period 202304.": "-- Enter SQL query\r\nSELECT TOP 5\r\n    B.[Proj:_Project_Name],\r\n    CAST(A.WIP_BALANCE AS FLOAT) AS wip_balance,\r\n    A.FISCPER\r\nFROM TPF_DUMP AS A\r\nJOIN BST10_ACTIVE_PROJECTS AS B\r\n    ON A.PROJECT_CODE = B.[Proj:_Project_Code]\r\nWHERE A.FISCPER = '202304'\r\nORDER BY CAST(A.WIP_BALANCE AS FLOAT) DESC;",
    "How many active projects are there?": "-- Enter SQL query\r\nSELECT COUNT(*) AS active_project_count FROM BST10_ACTIVE_PROJECTS;",
    "Show all project names and their start and end dates.": "-- Enter SQL query\r\nSELECT \r\n  [Proj:_Project_Name], \r\n  [Proj:_Start_Date], \r\n  [Proj:_End_Date]\r\nFROM BST10_ACTIVE_PROJECTS;",
    "Get the list of distinct fiscal periods available in the dataset.": "-- Enter SQL query\r\nSELECT DISTINCT FISCPER FROM TPF_DUMP ORDER BY FISCPER;",
    "Which projects had a TD variance below zero in fiscal period 202304?": "-- Enter SQL query\r\nSELECT \r\n    B.[Proj:_Project_Name], \r\n    A.TD_VARIANCE\r\nFROM TPF_DUMP A\r\nJOIN BST10_ACTIVE_PROJECTS B\r\n  ON A.PROJECT_CODE = B.[Proj:_Project_Code]\r\nWHERE A.FISCPER = '202304'\r\n  AND CAST(A.TD_VARIANCE AS FLOAT) < 0;",
    "List the top 3 clients by number of active projects.": "-- Enter SQL query\r\nSELECT TOP 3\r\n    [Proj:_Client_Name], \r\n    COUNT(*) AS project_count\r\nFROM BST10_ACTIVE_PROJECTS\r\nGROUP BY [Proj:_Client_Name]\r\nORDER BY project_count DESC;",
    "Find all projects in the 'Wastewater Treatment & Recycling' service line.": "-- Enter SQL query\r\nSELECT \r\n    [Proj:_Project_Name], \r\n    [Proj:_Service_Line]\r\nFROM BST10_ACTIVE_PROJECTS\r\nWHERE [Proj:_Service_Line] = 'Wastewater Treatment & Recycling';",
    "Show total R_BALANCE for fiscal period 202304.": "-- Enter SQL query\r\nSELECT \r\n    SUM(CAST(R_BALANCE AS FLOAT)) AS total_r_balance\r\nFROM TPF_DUMP\r\nWHERE FISCPER = '202304';",
    "Get average TD_EFF_EFFORT_AT_COMP across all projects for fiscal period 202305.": "-- Enter SQL query\r\nSELECT \r\n    AVG(CAST(TD_EFF_EFFORT_AT_COMP AS FLOAT)) AS avg_td_effort\r\nFROM TPF_DUMP\r\nWHERE FISCPER = '202305';",
    "Which projects had a WIP balance greater than 1000 in fiscal period 202304?": "-- Enter SQL query\r\nSELECT \r\n    B.[Proj:_Project_Name], \r\n    A.WIP_BALANCE\r\nFROM TPF_DUMP A\r\nJOIN BST10_ACTIVE_PROJECTS B\r\n  ON A.PROJECT_CODE = B.[Proj:_Project_Code]\r\nWHERE A.FISCPER = '202304'\r\n  AND CAST(A.WIP_BALANCE AS FLOAT) > 1000;",
    "What columns are available in the TPF_DUMP table?": "-- Enter SQL query\r\nSELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'TPF_DUMP';",
    "What columns are available in the BST10_ACTIVE_PROJECTS table?": "-- Enter SQL query\r\nSELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BST10_ACTIVE_PROJECTS';"
}
datasource.add_fewshots(json_key_pairs_dict)

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 31, Finished, Available, Finished)

In [25]:
datasource.get_fewshots()

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 32, Finished, Available, Finished)

,Id,Question,Query,State,Embedding
0,6515abc6-0e8b-4c8b-94b4-d9f169142e99,What is the total TD budget effort across all ...,-- Enter SQL query\r\nSELECT \r\n SUM(CAST(...,valid,"[-0.01423557, -0.03018208, -0.01094067, -0.001..."
1,dda97898-4401-4723-8d05-17b0f5c2443c,List project names and TD variance for all pro...,-- Enter SQL query\r\nSELECT \r\n B.[Proj:_...,valid,"[-0.016701661, -0.039200593, -0.019972978, -0...."
2,5fa7d059-bbc3-40a9-8c15-8ea8ab0bddd0,Show top 5 projects with the highest WIP balan...,-- Enter SQL query\r\nSELECT TOP 5\r\n B.[P...,valid,"[0.0050619026, -0.025896499, -0.016877612, -0...."
3,d70b4357-e5ea-4782-8d4b-2ebbe247d732,How many active projects are there?,-- Enter SQL query\r\nSELECT COUNT(*) AS activ...,valid,"[0.005137598, -0.022235071, -0.013972128, -0.0..."
4,099b734c-bd47-4223-a798-b205152843fa,Show all project names and their start and end...,-- Enter SQL query\r\nSELECT \r\n [Proj:_Proj...,valid,"[-0.012436671, -0.033227954, -0.011286891, -0...."
5,c8976d0e-6885-4fdf-b717-0bcdbbe10ef7,Get the list of distinct fiscal periods availa...,-- Enter SQL query\r\nSELECT DISTINCT FISCPER ...,valid,"[-0.015064417, -0.013549192, 0.0009918506, -0...."
6,7faf2cc8-a864-461b-8395-356ab3be88cc,Which projects had a TD variance below zero in...,-- Enter SQL query\r\nSELECT \r\n B.[Proj:_...,valid,"[-0.014941841, -0.028905671, -0.006995498, 0.0..."
7,3fe729de-4cd6-4644-93f0-b27b8bc55425,List the top 3 clients by number of active pro...,-- Enter SQL query\r\nSELECT TOP 3\r\n [Pro...,valid,"[0.014867043, -0.028551323, -0.014440699, -0.0..."
8,bc15b777-ceb4-4462-9f70-1cdb3c98c02b,Find all projects in the 'Wastewater Treatment...,-- Enter SQL query\r\nSELECT \r\n [Proj:_Pr...,valid,"[0.005156672, -0.024819333, -0.02101931, 0.000..."
9,de68c084-dd32-454f-8063-a147cc40cb46,Show total R_BALANCE for fiscal period 202304.,-- Enter SQL query\r\nSELECT \r\n SUM(CAST(...,valid,"[-0.012365327, -0.012888101, -0.014086993, -0...."


In [26]:
# # delete data agent
# delete_data_agent(data_agent_name)

StatementMeta(, 35254350-a682-408b-ac6f-4b32ab1e2d88, 33, Finished, Available, Finished)